In [1]:
# Import necessary libraries
import requests
import json
import uuid
import time
import asyncio
import aiohttp

# Define the base URL of the API
BASE_URL = "http://localhost:8081"

async def create_chat(session, user_id):
    url = f"{BASE_URL}/chats/"
    headers = {"user-id": user_id}
    payload = {"user_id": user_id, "session_id": str(uuid.uuid4())}
    async with session.post(url, headers=headers, json=payload) as response:
        if response.status == 200:
            print(f"Chat created successfully for user {user_id}!")
            return await response.json()
        else:
            print(f"Failed to create chat for user {user_id}: {response.status} - {await response.text()}")
            return None

async def get_chats(session, user_id, skip=0, limit=10):
    url = f"{BASE_URL}/chats/?skip={skip}&limit={limit}"
    headers = {"user-id": user_id}
    async with session.get(url, headers=headers) as response:
        if response.status == 200:
            return await response.json()
        else:
            print(f"Failed to get chats for user {user_id}: {response.status} - {await response.text()}")
            return None

async def get_chat(session, user_id, chat_id):
    url = f"{BASE_URL}/chats/{chat_id}"
    headers = {"user-id": user_id}
    async with session.get(url, headers=headers) as response:
        if response.status == 200:
            return await response.json()
        else:
            print(f"Failed to get chat for user {user_id}: {response.status} - {await response.text()}")
            return None

async def delete_chat(session, user_id, chat_id):
    url = f"{BASE_URL}/chats/{chat_id}"
    headers = {"user-id": user_id}
    async with session.delete(url, headers=headers) as response:
        if response.status == 200:
            print(f"Chat {chat_id} deleted successfully for user {user_id}!")
        else:
            print(f"Failed to delete chat for user {user_id}: {response.status} - {await response.text()}")

async def add_message(session, user_id, chat_id, role, content):
    url = f"{BASE_URL}/chats/{chat_id}/messages/"
    headers = {"user-id": user_id}
    payload = {"role": role, "content": content}
    async with session.post(url, headers=headers, json=payload) as response:
        if response.status == 200:
            print(f"Message added to chat {chat_id} for user {user_id}!")
            return await response.json()
        else:
            print(f"Failed to add message for user {user_id}: {response.status} - {await response.text()}")
            return None

async def get_messages(session, user_id, chat_id):
    chat = await get_chat(session, user_id, chat_id)
    if chat:
        return chat.get("messages", [])
    else:
        print(f"Failed to get messages for chat {chat_id} of user {user_id}")
        return []

async def list_all_messages(session, all_chats):
    for user, chats in all_chats:
        for chat_id in chats:
            messages = await get_messages(session, user, chat_id)
            print(f"Messages for chat {chat_id} of user {user}:")
            for message in messages:
                print(f"  {message['role']}: {message['content']}")

async def list_all_chats(session, users):
    for user in users:
        chats = await get_chats(session, user)
        if chats:
            print(f"Chats for user {user}:")
            for chat in chats:
                print(f"  Chat ID: {chat['_id']}, Session ID: {chat['session_id']}")

async def delete_all_chats(session, all_chats):
    for user, chats in all_chats:
        for chat_id in chats:
            await delete_chat(session, user, chat_id)


In [2]:
async def simulate_users_activity(num_users=5, num_chats_per_user=3, num_messages_per_chat=2):
    users = [f"user_{i}@example.com" for i in range(num_users)]
    all_chats = []

    async with aiohttp.ClientSession() as session:
        for user in users:
            print("Current user: ", user)
            user_chats = []
            for _ in range(num_chats_per_user):
                chat = await create_chat(session, user)
                if chat:
                    chat_id = chat.get("_id")
                    for n in range(num_messages_per_chat):
                        await add_message(session, user, chat_id, "user", f"Hello number {n} from {user}!")
                    user_chats.append(chat_id)
            all_chats.append((user, user_chats))
        return all_chats

all_chats = await simulate_users_activity(num_users=3, num_chats_per_user=2, num_messages_per_chat=3)
print("Simulation completed. Chats created and messages added.")


Current user:  user_0@example.com
Chat created successfully for user user_0@example.com!
Message added to chat 666991b4e207611a154c0c7f for user user_0@example.com!
Message added to chat 666991b4e207611a154c0c7f for user user_0@example.com!
Message added to chat 666991b4e207611a154c0c7f for user user_0@example.com!
Chat created successfully for user user_0@example.com!
Message added to chat 666991b4e207611a154c0c80 for user user_0@example.com!
Message added to chat 666991b4e207611a154c0c80 for user user_0@example.com!
Message added to chat 666991b4e207611a154c0c80 for user user_0@example.com!
Current user:  user_1@example.com
Chat created successfully for user user_1@example.com!
Message added to chat 666991b4e207611a154c0c81 for user user_1@example.com!
Message added to chat 666991b4e207611a154c0c81 for user user_1@example.com!
Message added to chat 666991b4e207611a154c0c81 for user user_1@example.com!
Chat created successfully for user user_1@example.com!
Message added to chat 666991

In [3]:
print("Listing all chats before deletion:")
await list_all_chats(aiohttp.ClientSession(), [f"user_{i}@example.com" for i in range(3)])

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd895982cc0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fd89597e0f0>, 694.428053853)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fd895983ec0>


Listing all chats before deletion:
Chats for user user_0@example.com:
  Chat ID: 666991b4e207611a154c0c7f, Session ID: 2e542bd7-4619-4e3a-a6c6-487dc502c69d
  Chat ID: 666991b4e207611a154c0c80, Session ID: 00c4f89c-7e6c-47bb-b08c-e18001b5f3ad
Chats for user user_1@example.com:
  Chat ID: 666991b4e207611a154c0c81, Session ID: f585d1c9-a7c9-46fe-8a31-02a9486378f0
  Chat ID: 666991b4e207611a154c0c82, Session ID: a9367053-170e-4ded-835b-f5419667478e
Chats for user user_2@example.com:
  Chat ID: 666991b4e207611a154c0c83, Session ID: 48c3e30c-3229-4f5d-a5dd-4b24e0727140
  Chat ID: 666991b4e207611a154c0c84, Session ID: d7161226-0911-4621-8f42-4b01b2e47d6d


In [4]:
await list_all_messages(aiohttp.ClientSession(), all_chats)


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd895c82120>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fd89597deb0>, 697.384601256)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fd8959a4560>


Messages for chat 666991b4e207611a154c0c7f of user user_0@example.com:
  user: Hello number 0 from user_0@example.com!
  assistant: <p>Hello! How can I assist you today?</p>
  user: Hello number 1 from user_0@example.com!
  assistant: <p>Hello! How can I assist you today?</p>
  user: Hello number 2 from user_0@example.com!
  assistant: <p>Hello! How can I assist you today?</p>
Messages for chat 666991b4e207611a154c0c80 of user user_0@example.com:
  user: Hello number 0 from user_0@example.com!
  assistant: <p>Hello! How can I assist you today?</p>
  user: Hello number 1 from user_0@example.com!
  assistant: <p>Hello! How can I assist you today?</p>
  user: Hello number 2 from user_0@example.com!
  assistant: <p>Hello! How can I assist you today?</p>
Messages for chat 666991b4e207611a154c0c81 of user user_1@example.com:
  user: Hello number 0 from user_1@example.com!
  assistant: <p>Hello! How can I assist you today?</p>
  user: Hello number 1 from user_1@example.com!
  assistant: <p>He

In [5]:
await delete_all_chats(aiohttp.ClientSession(), all_chats)
print("All chats deleted.")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f1a1d166990>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f1a1d1621b0>, 12659.824748551)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f1a2c115940>


Chat 6666cafcdbc13253825c23d0 deleted successfully for user user_0@example.com!
Chat 6666cafcdbc13253825c23d1 deleted successfully for user user_0@example.com!
Chat 6666cafcdbc13253825c23d2 deleted successfully for user user_1@example.com!
Chat 6666cafcdbc13253825c23d3 deleted successfully for user user_1@example.com!
Chat 6666cafcdbc13253825c23d4 deleted successfully for user user_2@example.com!
Chat 6666cafcdbc13253825c23d5 deleted successfully for user user_2@example.com!
All chats deleted.


In [6]:
print("Listing all chats after deletion:")
await list_all_chats(aiohttp.ClientSession(), [f"user_{i}@example.com" for i in range(3)])

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f1a1d1895e0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f1a1d162510>, 12659.857007143)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f1a1d189340>


Listing all chats after deletion:
